# Sort out Academic agenda and plan classes

In this notebook, you can import your agenda in `ics` files and inspect proposed classes schedule (typically in XLS files) for next semester. 

As a teacher in Ecole Centrale de Nantes, this version deals with EI1 weekly schedule and options/small groups schedules.

In [1]:
# Import necessary libraries
import pandas as pd
from icalendar import Calendar
from datetime import datetime, time
from datetime import datetime, timedelta
import calendar
import pytz

## Step 1: Import ICS files

In [223]:
time.max

datetime.time(23, 59, 59, 999999)

In [249]:
# Function to read and parse an ICS file
def read_ics(file_path):
    with open(file_path, 'r') as file:
        gcal = Calendar.from_ical(file.read())
    utc=pytz.UTC
    now = datetime.now().replace(tzinfo=utc)
    events = []
    for component in gcal.walk():
        if component.name == "VEVENT":
           # try:
                dtstart=component.get('dtstart').dt
                if isinstance(dtstart, datetime):
                    if not dtstart.tzinfo:
                        dtstart = dtstart.replace(tzinfo=pytz.UTC)
                else:  # if dtstart is a date object
                    dtstart = datetime.combine(dtstart, time.min, tzinfo=pytz.UTC)
                if dtstart < now: continue
            
                dtend = component.get('dtend').dt if component.get('dtend') else dtstart
                if isinstance(dtend, datetime):
                    if not dtend.tzinfo:
                            dtend = dtend.replace(tzinfo=pytz.UTC)
                else:  # if dtend is a date object
                    print(component.get('summary'), dtend)
                    dtend = datetime.combine(dtend, time.max, tzinfo=pytz.UTC)
                        
                event = {
                    'summary': component.get('summary'),
                    'dtstart': dtstart,
                    'dtend': dtend,
                    'location': component.get('location'),
                    'description': component.get('description')
                }
                
                events.append(event)
            #except:
            #    print("Data ingnored: some error occured in:", component)

        # Filter events to only include those after now
    return pd.DataFrame(events)

In [250]:
file_path="../ECN.ics"
ecn_df=read_ics(file_path)

https://www.wccm2024.org/ 2024-07-27
CA Audition des candidats Direction 2025-04-04


# Step 2 reading excel files

## Step 2.1 reading EI1 xls file

Albeit annoying, the formatting of the agenda is fixed, so it's fairly easy to obtain the schedule of a given course for a given group.

It is build as follow:
- each sheet is a week named accordingly
- each group is a row
- columns correspond to time slots from Mon M1("C) to Fri S2 ("V")
- Each cell is either void or contains the course short name e.g. FLUID, followed by course type (TP, TD, CM)

Thus, the extraction function needs to know the structure and find course short name occurences on all sheets and output a dataframe with the same structure as ICS pandas imports.

Because of numerous merged cells, it is better for us to rely on `openpyxl` directly. to extract course times and groups. These functions will then be converted into some helper script for yearly analysis.

In [4]:
class_slots={"M1": ["08:00","10:00"],"M2": ["08:00","10:00"],"S1": ["10:15","15:45"],"S2": ["16:00","18:00"]}
class_slots

{'M1': ['08:00', '10:00'],
 'M2': ['08:00', '10:00'],
 'S1': ['10:15', '15:45'],
 'S2': ['16:00', '18:00']}

In [5]:
file_path="../general/EDTs24-25/ET_EI1S5 _2024-2025_VF.xlsx"

In [119]:
#https://www.reddit.com/r/excel/comments/10w12bt/is_there_a_way_to_unmerge_cells_and_automatically/
import openpyxl


def create_merged_cell_lookup(sheet) -> dict:
    """
    Creates a lookup dictionary for merged cells in a given sheet.
    
    This function iterates through all merged cell ranges in the given sheet,
    and creates a dictionary where the keys are the merged cell ranges (as strings)
    and the values are the values of the top-left cell in each merged cell range.

    Args:
        sheet (openpyxl.worksheet.worksheet.Worksheet): The worksheet object to process.

    Returns:
        dict: A dictionary with merged cell ranges as keys and the top-left cell values as values.
    """
    merged_lookup = {}
    for cell_group in sheet.merged_cells.ranges:
        min_col, min_row, max_col, max_row = openpyxl.utils.range_boundaries(str(cell_group))
        #if min_col == max_col:
        top_left_cell_value = sheet.cell(row=min_row, column=min_col).value
        merged_lookup[str(cell_group)] = top_left_cell_value
    return merged_lookup

 
def unmerge_cell_copy_top_value(workbook_path: str, output_save="", verbose: bool=False):
    """
    Unmerges cells in the given workbook and copies the top-left cell value to all cells in each previously merged range.
    
    This function opens the workbook at the given path, processes each worksheet by unmerging all merged cells,
    and copies the value of the top-left cell in each merged range to all cells in that range. The modified workbook
    is saved as "ready4Import.xlsx" in the current working directory.

    Args:
        workbook_path (str): The path to the Excel workbook to process.
        output_save (str): if not empty, will save the file to requested ouput.
        verbose (bool): If True, print debug information during processing. Default is False.

    Returns:
        openpyxl.workbook.workbook.Workbook: The modified workbook object.
    """
    wbook = openpyxl.load_workbook(workbook_path, data_only=True)
    
    for sheet in wbook.worksheets:
        lookup = create_merged_cell_lookup(sheet)
        if verbose: print(lookup)
        cell_group_list = lookup.keys()
        for cell_group in cell_group_list:
            min_col, min_row, max_col, max_row = openpyxl.utils.range_boundaries(str(cell_group))
            sheet.unmerge_cells(str(cell_group))
            if verbose: print(min_col, min_row, max_col, max_row)
            for row in sheet.iter_rows(min_col=min_col, min_row=min_row, max_col=max_col, max_row=max_row):
                if verbose :print(lookup[cell_group])
                for cell in row:
                    cell.value = lookup[cell_group]
                    if verbose: print(cell.coordinate)
    if output_save : 
        wbook.save(output_save)
    return wbook

def search_string_in_workbook(wbook: "openpyxl workbook", search_string: str):
    """
    Searches for a specified string in all sheets of a workbook and extracts their coordinates along with sheet names and cell values.

    This function iterates through each worksheet in the provided openpyxl workbook, searches for the specified string in all cells,
    and stores the results, including the sheet name, cell coordinate, and cell value.

    Args:
        wbook (openpyxl.workbook.workbook.Workbook): The openpyxl workbook object to process.
        search_string (str): The string to search for in the workbook.

    Returns:
        list: A list of lists, where each inner list contains the sheet name, cell coordinate, and cell value of a match.
    """
    search_results=[]
    for sheet in wbook.worksheets:
        search_results+=search_string_in_worksheet(sheet,search_string)
    
    return search_results


def search_string_in_worksheet(sheet, search_string):
    search_results=[]
    for row in sheet.iter_rows():
            for cell in row:
                if cell.value and isinstance(cell.value, str):
                    if search_string in cell.value:
                        search_results.append([sheet.title,cell.coordinate, cell.value])
    return search_results


In [128]:
def extract_schedule_by_group_EI1(file_path,course_name,group_name,course_type, display_group_schedule=False):
    """
    This function is specific to the design of EI1 course schedule xlsx file at ECN
    """
    class_slots={"M1": ["08:00","10:00"],"M2": ["10:15","12:15"],"S1": ["13:45","15:45"],"S2": ["16:00","18:00"]}
    tz = pytz.timezone('Europe/Paris')

    events=[]
    wbook=unmerge_cell_copy_top_value(file_path)
    
    #finding group row using column B
    for row in wbook["40"].iter_rows(min_col=2, min_row=4, max_col=2, max_row=20):
        for cell in row:
            if cell.value and isinstance(cell.value, str):
                if group_name in cell.value:
                    groups_row=str(cell.row)
                    
    occurences=search_string_in_workbook(wbook , search_string=course_name)
    for occ in occurences:
        if occ[1][1:] != groups_row or occ[2].split()[1][:2] not in course_type: continue # Beware very hacky test
        # extracting date
        date_pos=occ[1][0]+"3"
        date=wbook[occ[0]][date_pos].value.split()[1] # dropping day name
        slot_pos=occ[1][0]+"4"
        slot=wbook[occ[0]][slot_pos].value.strip()
        
        dtstart= tz.localize(datetime.strptime(date+" "+class_slots[slot][0],'%d/%m/%y %H:%M'))#converting to date object
        dtend= tz.localize(datetime.strptime(date+" "+class_slots[slot][1],'%d/%m/%y %H:%M'))#converting to date object

        if display_group_schedule: 
            print(f"{wbook[occ[0]][date_pos].value.split()[0]:<8} {date} {slot} {occ[2]}")
            
        events.append({ 'summary': occ[2],
                        'dtstart': dtstart,
                        'dtend': dtend})
    return events

In [129]:
EI1_events=extract_schedule_by_group_EI1(file_path,"ALGO","B",["TP","TD", "CM"],True)

VENDREDI 27/09/24 M2 ALGO CM1
VENDREDI 27/09/24 S2 ALGO TD 1
VENDREDI 04/10/24 S2 ALGO CM2
VENDREDI 11/10/24 S1 ALGO TD 2
LUNDI    04/11/24 S2 ALGO TD 3
MERCREDI 13/11/24 M2 ALGO TD 4
MERCREDI 20/11/24 S2 ALGO TD 5
JEUDI    28/11/24 M2 ALGO CM3
MERCREDI 04/12/24 M1 ALGO TD 6
VENDREDI 13/12/24 S1 ALGO TP1
VENDREDI 13/12/24 S2 ALGO TP1
VENDREDI 20/12/24 S1 ALGO TP2
VENDREDI 20/12/24 S2 ALGO TP2
VENDREDI 10/01/25 M1 ALGO TP3
VENDREDI 10/01/25 M2 ALGO TP3


In [75]:
# export this data in the same format as ICS extractions to facilitate review

## Step 2.2 : read Option schedule and BBA

Both file follow the same format where a single sheet represents the whole semester. There is only one group (for now during other years, we had a label in the text to know which group does what).
A specific column gives the date of the first day of the week. It has to be given by the user.. Then each slot is organized as in EI1 from monday M1 to friday S2. 

In [191]:
def extract_schedule_1sheet_format(file_path, 
                                   course_name, 
                                   date_column, 
                                   line_slot="2", 
                                   group_name=None, 
                                   course_type=None, 
                                   display_group_schedule=False):
    """
    Extracts the schedule for a specific course from a single-sheet formatted course schedule Excel file.
    
    Args:
        file_path (str): The path to the Excel file containing the course schedule.
        course_name (str): The name of the course to search for in the schedule.
        date_column (str): The column containing the dates in the schedule.
        line_slot (str, optional): The row number in the sheet to look for class slots (default is "2").
        group_name (str, optional): The group name (not used in current implementation).
        course_type (str, optional): The type of the course (not used in current implementation).
        display_group_schedule (bool, optional): If True, displays the schedule for the group (default is False).

    Returns:
        list: A list of dictionaries, each representing an event with keys 'summary', 'dtstart', and 'dtend'.
    
    Notes:
        This function is specific to the design of one-sheet course schedule xlsx files at ECN. 
        It assumes that the schedule is structured in a specific way with class slots defined as:
            - M1: 08:00-10:00
            - M2: 10:15-12:15
            - S1: 13:45-15:45
            - S2: 16:00-18:00
        The function handles merged cells and extracts the necessary information based on the course name.

        For successive courses, you can add a blank space at the end of the course name in order to disentangle these.
    """
    class_slots={"M1": [[ 8, 0],[10, 0]],
                 "M2": [[10,15],[12,15]],
                 "S1": [[13,45],[15,45]],
                 "S2": [[16, 0],[18, 0]]}
    tz = pytz.timezone('Europe/Paris')

    events=[]
    wbook=unmerge_cell_copy_top_value(file_path)
                    
    occurences=search_string_in_workbook(wbook, search_string=course_name)
    for occ in occurences:
        #if occ[2].split()[1][:2] not in course_type: continue # Beware very hacky test
        # extracting date
        #print(occ)
        date_pos=date_column+"".join(filter(lambda x: x.isdigit(), occ[1]))
        date=wbook[occ[0]][date_pos].value # dropping day name
        slot_pos="".join(filter(lambda x: x.isalpha(), occ[1]))+line_slot
        #print(date_pos,slot_pos)
        slot=wbook[occ[0]][slot_pos].value.strip()
        dtstart= tz.localize(date.replace(hour=class_slots[slot][0][0],minute=class_slots[slot][0][1]))#converting to date object
        dtend=   tz.localize(date.replace(hour=class_slots[slot][1][0],minute=class_slots[slot][1][1]))#converting to date object

        if display_group_schedule: 
            print(date.strftime("%a %d/%m/%y"),f"{slot} {occ[2]}")
            
        events.append({ 'summary': occ[2],
                        'dtstart': dtstart,
                        'dtend': dtend})
    return events

In [192]:
INFOIA=extract_schedule_1sheet_format("../general/EDTs24-25/Fichier_type_24_25.INFO IA.xlsx", "PAPY", "G",line_slot="2",display_group_schedule=True)

Mon 02/09/24 M2 PAPY
 CM - LL
Mon 02/09/24 S1 PAPY
 TP – LL
Mon 02/09/24 S2 PAPY
 TP – LL
Mon 09/09/24 M1 PAPY
 CM - LL
Mon 09/09/24 M2 PAPY
 TP – LL
Mon 23/09/24 M1 PAPY
 TP – LL
Mon 23/09/24 M2 PAPY
 TP – LL
Mon 30/09/24 M1 PAPY
 CM - LL
Mon 30/09/24 M2 PAPY
 TP – LL
Mon 07/10/24 M1 PAPY
 TP – LL
Mon 07/10/24 M2 PAPY
 TP – LL
Mon 14/10/24 M1 PAPY
 CM - LL
Mon 14/10/24 M2 PAPY
 TP – LL
Mon 21/10/24 M1 PAPY
 TP – LL
Mon 21/10/24 M2 PAPY
 TP – LL
Mon 04/11/24 M1 PAPY
DS – LL


In [193]:
BBA=extract_schedule_1sheet_format("../general/EDTs24-25/BBA2_24_25.xlsx", "INFO V ", "C",display_group_schedule=True)

Mon 16/09/24 S1 INFO V - CM   LL
Mon 23/09/24 S1 INFO V - TP   LL  +  ?
Mon 30/09/24 S1 INFO V - CM   LL
Mon 07/10/24 S1 INFO V - TP   LL  +  ?
Mon 14/10/24 S1 INFO V - CM   LL
Mon 21/10/24 S1 INFO V - TP   LL  + ?
Mon 04/11/24 S1 INFO V - CM   LL
Mon 11/11/24 S1 INFO V - TP   LL  +  ?
Mon 18/11/24 S1 INFO V - CM   LL
Mon 25/11/24 S1 INFO V - TP   LL  +  ?
Mon 02/12/24 S1 INFO V - CM   LL
Mon 09/12/24 S1 INFO V - TP   LL  + ?
Mon 16/12/24 M2 INFO V - DS   LL


# Step 3: report conflict and export complete schedule for vizualtion

In [228]:
combined_df = pd.concat([ecn_df,pd.DataFrame(BBA), pd.DataFrame(INFOIA),pd.DataFrame(EI1_events)], ignore_index=True)
#combined_df = pd.concat([pd.DataFrame(BBA), pd.DataFrame(INFOIA),pd.DataFrame(EI1_events)], ignore_index=True)

In [229]:
combined_df

,summary,dtstart,dtend,location,description
0,https://www.wccm2024.org/,2024-07-21 00:00:00+00:00,2024-07-27 23:59:59.999999+00:00,None,None
1,CA,2024-10-17 13:30:00+02:00,2024-10-17 23:59:59.999999+00:00,None,None
2,Jurys BBA 3,2024-09-02 13:30:00+02:00,2024-09-02 23:59:59.999999+00:00,Audencia,\n
3,Jurys BBA 3,2024-09-02 13:30:00+02:00,2024-09-02 23:59:59.999999+00:00,Audencia,\n
4,CAR,2024-12-09 15:00:00+01:00,2024-12-09 23:59:59.999999+00:00,None,None
...,...,...,...,...,...
70,ALGO TP1,2024-12-13 16:00:00+01:00,2024-12-13 18:00:00+01:00,NaN,NaN
71,ALGO TP2,2024-12-20 13:45:00+01:00,2024-12-20 15:45:00+01:00,NaN,NaN
72,ALGO TP2,2024-12-20 16:00:00+01:00,2024-12-20 18:00:00+01:00,NaN,NaN
73,ALGO TP3,2025-01-10 08:00:00+01:00,2025-01-10 10:00:00+01:00,NaN,NaN


In [230]:
# Sort the DataFrame by the 'dtstart' column
combined_df = combined_df.sort_values(by='dtstart').reset_index(drop=True)
combined_df

,summary,dtstart,dtend,location,description
0,CAR?,2024-07-03 09:00:00+02:00,2024-07-03 23:59:59.999999+00:00,None,None
1,Commissions de discipline HUBERT-DOUMERC et MA...,2024-07-04 09:00:00+02:00,2024-07-04 23:59:59.999999+00:00,None,"Bonjour,\r\nJe vous propose de retenir le *4 j..."
2,Jury d'année BBA1 -BBA2 -BB3,2024-07-11 14:00:00+02:00,2024-07-11 23:59:59.999999+00:00,Centrale Nantes T243,None
3,Vol N°AF7509 de NTE à CDG - ref:VPFG3H pour LE...,2024-07-13 06:05:00+02:00,2024-07-13 23:59:59.999999+00:00,Aéroport Nantes Atlantique - Nantes - France->...,Attention : Les dates et horaires de ce rendez...
4,Vol N°AF374 de CDG à YVR - ref:VPFG3H pour LES...,2024-07-13 10:10:00+02:00,2024-07-13 23:59:59.999999+00:00,Aéroport Paris–Charles de Gaulle - Paris - Fra...,Attention : Les dates et horaires de ce rendez...
...,...,...,...,...,...
70,Vote,2025-04-04 09:00:00+02:00,2025-04-04 23:59:59.999999+00:00,None,None
71,CAR,2025-05-28 10:00:00+02:00,2025-05-28 23:59:59.999999+00:00,None,None
72,CA,2025-06-26 13:30:00+02:00,2025-06-26 23:59:59.999999+00:00,None,None
73,CA,2025-10-09 13:30:00+02:00,2025-10-09 23:59:59.999999+00:00,None,None


In [251]:
# Create a list to store conflicting events
conflicts = []

# Iterate through the sorted DataFrame to identify conflicts
for i in range(1, len(combined_df)):
    current_event = combined_df.iloc[i]
    previous_event = combined_df.iloc[i - 1]
    
    if current_event['dtstart'] < previous_event['dtend']:
        conflicts.append({
            'event1_summary': previous_event['summary'],
            'event1_dtstart': previous_event['dtstart'],
            'event1_dtend': previous_event['dtend'],
            'event2_summary': current_event['summary'],
            'event2_dtstart': current_event['dtstart'],
            'event2_dtend': current_event['dtend']
        })

# Convert the list of conflicts to a DataFrame
conflicts_df = pd.DataFrame(conflicts)

In [254]:
conflicts_df

,event1_summary,event1_dtstart,event1_dtend,event2_summary,event2_dtstart,event2_dtend
0,Vol N°AF7509 de NTE à CDG - ref:VPFG3H pour LE...,2024-07-13 06:05:00+02:00,2024-07-13 23:59:59.999999+00:00,Vol N°AF374 de CDG à YVR - ref:VPFG3H pour LES...,2024-07-13 10:10:00+02:00,2024-07-13 23:59:59.999999+00:00
1,https://www.wccm2024.org/,2024-07-21 00:00:00+00:00,2024-07-27 23:59:59.999999+00:00,Vol N°AF375 de YVR à CDG - ref:VPFG3H pour LES...,2024-07-27 13:30:00+02:00,2024-07-28 23:59:59.999999+00:00
2,Vol N°AF375 de YVR à CDG - ref:VPFG3H pour LES...,2024-07-27 13:30:00+02:00,2024-07-28 23:59:59.999999+00:00,Air France- AF7771- Paris 7/28/2024 9:45:00 A...,2024-07-28 07:45:00+00:00,2024-07-28 23:59:59.999999+00:00
3,Air France- AF7771- Paris 7/28/2024 9:45:00 A...,2024-07-28 07:45:00+00:00,2024-07-28 23:59:59.999999+00:00,AF 7771 de Aéroport Paris–Charles de Gaulle - ...,2024-07-28 09:45:00+02:00,2024-07-28 23:59:59.999999+00:00
4,Jurys BBA1,2024-09-02 09:00:00+02:00,2024-09-02 23:59:59.999999+00:00,PAPY\n CM - LL,2024-09-02 10:15:00+02:00,2024-09-02 12:15:00+02:00
5,Jurys BBA 3,2024-09-02 13:30:00+02:00,2024-09-02 23:59:59.999999+00:00,Jurys BBA 3,2024-09-02 13:30:00+02:00,2024-09-02 23:59:59.999999+00:00
6,Jurys BBA 3,2024-09-02 13:30:00+02:00,2024-09-02 23:59:59.999999+00:00,PAPY\n TP – LL,2024-09-02 13:45:00+02:00,2024-09-02 15:45:00+02:00
7,INFO V - CM LL,2024-10-14 13:45:00+02:00,2024-10-14 15:45:00+02:00,CAR,2024-10-14 15:00:00+02:00,2024-10-14 23:59:59.999999+00:00
8,Matinée séminaire CA,2024-10-17 09:30:00+02:00,2024-10-17 23:59:59.999999+00:00,CA,2024-10-17 13:30:00+02:00,2024-10-17 23:59:59.999999+00:00
9,INFO V - TP LL + ?,2024-12-09 13:45:00+01:00,2024-12-09 15:45:00+01:00,CAR,2024-12-09 15:00:00+01:00,2024-12-09 23:59:59.999999+00:00
